<a href="https://colab.research.google.com/github/METATRONY/network-kpi-forecasting/blob/main/Aura_Ring_Backend_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
import time
import subprocess
import platform

# Attempt to import pybluez for Bluetooth discovery.
# If pybluez is not installed or available, device discovery will be simulated.
try:
    import bluetooth
    BLUETOOTH_AVAILABLE = True
except ImportError:
    print("Warning: 'pybluez' library not found. Bluetooth device discovery will be simulated.")
    print("To install pybluez: pip install pybluez")
    BLUETOOTH_AVAILABLE = False

# --- Configuration ---
# Define RSSI sensitivity levels and their corresponding thresholds (in dBm).
# RSSI values are typically negative; a value closer to 0 indicates a stronger signal.
# The screen will lock when the RSSI reading is BELOW (more negative than) the threshold.
# For example, if sensitivity is 'High' (-60 dBm), the screen locks if RSSI drops to -61, -70, etc.
RSSI_SENSITIVITY_LEVELS = {
    "1 (Very Low - locks far away)": -90,  # Locks when signal is very weak (far away)
    "2 (Low)": -80,
    "3 (Medium)": -70,
    "4 (High)": -60,
    "5 (Very High - locks very close)": -50 # Locks when signal is relatively strong (close)
}

# Polling interval for RSSI readings (in seconds)
POLLING_INTERVAL = 2

# --- OS-Specific Screen Lock Commands ---
# IMPORTANT: Uncomment the command for your operating system.
# You might need to install additional tools or configure permissions for these to work.
def lock_screen():
    """Locks the screen based on the operating system."""
    system_os = platform.system()
    print(f"\n--- RSSI below threshold! Locking screen on {system_os} ---")

    try:
        if system_os == "Windows":
            # Windows: Locks the workstation
            # Requires 'ctypes' for more robust locking or 'rundll32.exe user32.dll,LockWorkStation'
            # For simplicity, using a common command.
            subprocess.run(["rundll32.exe", "user32.dll,LockWorkStation"], check=True)
        elif system_os == "Darwin":
            # macOS: Locks the screen using 'pmset' or 'osascript'
            # 'pmset displaysleepnow' puts displays to sleep, which often triggers lock
            # 'osascript -e 'tell application "System Events" to keystroke "q" using {command down, control down}''
            # is another common method for locking.
            subprocess.run(["pmset", "displaysleepnow"], check=True)
        elif system_os == "Linux":
            # Linux: Varies greatly by desktop environment (GNOME, KDE, XFCE, etc.)
            # Examples for common desktop environments:
            # GNOME: 'gnome-screensaver-command --lock' or 'dbus-send --type=method_call --dest=org.gnome.ScreenSaver /org/gnome/ScreenSaver org.gnome.ScreenSaver.Lock'
            # KDE: 'qdbus org.kde.ScreenSaver /ScreenSaver Lock'
            # XFCE: 'xflock4'
            # Unity/Cinnamon: 'dm-tool lock'
            # You might need to install the screensaver package for your environment.
            # Example for GNOME (uncomment if using GNOME):
            # subprocess.run(["gnome-screensaver-command", "--lock"], check=True)
            # Generic X11 lock (might require 'xss-lock' or similar):
            subprocess.run(["xdg-screensaver", "lock"], check=True)
        else:
            print(f"Error: Screen locking not supported on {system_os}.")
            return False
        print("Screen lock command executed.")
        return True
    except FileNotFoundError as e:
        print(f"Error: Screen lock command not found. Please ensure it's installed and in your PATH. ({e})")
        return False
    except subprocess.CalledProcessError as e:
        print(f"Error executing screen lock command: {e}")
        return False
    except Exception as e:
        print(f"An unexpected error occurred during screen lock: {e}")
        return False

# --- Bluetooth Device Discovery (Conceptual/Simulated) ---
def discover_bluetooth_devices():
    """
    Discovers nearby Bluetooth devices.
    Note: pybluez's discover_devices is for inquiry, not for listing connected devices.
    For a real application, you'd need a more robust way to identify and monitor
    a *specific connected* Aura Ring.
    """
    print("\nSearching for Bluetooth devices (this may take a few seconds)...")
    if BLUETOOTH_AVAILABLE:
        try:
            # Duration in seconds to scan for devices
            nearby_devices = bluetooth.discover_devices(duration=8, lookup_names=True, flush_cache=True, lookup_class=False)
            if not nearby_devices:
                print("No Bluetooth devices found. Please ensure Bluetooth is enabled and devices are discoverable.")
                # Simulate some devices if none are found for demonstration
                return [("AA:BB:CC:DD:EE:F1", "Simulated Aura Ring 1"),
                        ("AA:BB:CC:DD:EE:F2", "Simulated Bluetooth Device")]
            return nearby_devices
        except bluetooth.BluetoothError as e:
            print(f"Bluetooth error during discovery: {e}")
            print("Falling back to simulated devices.")
            return [("AA:BB:CC:DD:EE:F1", "Simulated Aura Ring 1"),
                    ("AA:BB:CC:DD:EE:F2", "Simulated Bluetooth Device")]
    else:
        print("Bluetooth library not available, simulating device discovery.")
        # Return some dummy devices for demonstration if pybluez is not installed
        return [("AA:BB:CC:DD:EE:F1", "Simulated Aura Ring 1"),
                ("AA:BB:CC:DD:EE:F2", "Simulated Bluetooth Device")]

def select_device(devices):
    """Allows the user to select a Bluetooth device from a list."""
    if not devices:
        print("No devices to select. Exiting.")
        sys.exit(1)

    print("\nFound the following Bluetooth devices:")
    for i, (addr, name) in enumerate(devices):
        print(f"{i + 1}. {name} ({addr})")

    while True:
        try:
            choice = int(input("Enter the number of your Aura Ring device: "))
            if 1 <= choice <= len(devices):
                selected_addr, selected_name = devices[choice - 1]
                print(f"Selected Aura Ring: {selected_name} ({selected_addr})")
                return selected_addr, selected_name
            else:
                print("Invalid choice. Please enter a number from the list.")
        except ValueError:
            print("Invalid input. Please enter a number.")

def select_sensitivity():
    """Allows the user to select an RSSI sensitivity level."""
    print("\nSelect RSSI Sensitivity Level (screen locks when signal is weaker than this):")
    sensitivity_options = list(RSSI_SENSITIVITY_LEVELS.keys())
    for i, level_desc in enumerate(sensitivity_options):
        print(f"{i + 1}. {level_desc} (Threshold: {RSSI_SENSITIVITY_LEVELS[level_desc]} dBm)")

    while True:
        try:
            choice = int(input("Enter the number for your desired sensitivity: "))
            if 1 <= choice <= len(sensitivity_options):
                selected_level_desc = sensitivity_options[choice - 1]
                threshold = RSSI_SENSITIVITY_LEVELS[selected_level_desc]
                print(f"Selected sensitivity: {selected_level_desc} (Threshold: {threshold} dBm)")
                return threshold
            else:
                print("Invalid choice. Please enter a number from the list.")
        except ValueError:
            print("Invalid input. Please enter a number.")

# --- RSSI Monitoring Loop (Conceptual/Simulated) ---
def get_current_rssi(device_address):
    """
    Conceptual function to get current RSSI.
    In a real scenario, this would involve continuous polling of the connected device's RSSI property.
    For this demonstration, it will simulate fluctuating RSSI.
    """
    # This is a placeholder. In a real application, you'd query the OS Bluetooth stack.
    # Example for pybluez (often only works during discovery, not for connected devices):
    # try:
    #     sock = bluetooth.BluetoothSocket(bluetooth.RFCOMM)
    #     sock.connect((device_address, 1)) # Connect to a service (e.g., port 1)
    #     # This part is highly dependent on how the device exposes RSSI
    #     # Some devices might provide it via a custom service or characteristic (BLE)
    #     # For classic BT, it's often not directly exposed via simple socket calls.
    #     # You'd need to delve into BlueZ D-Bus API on Linux, CoreBluetooth on macOS, etc.
    #     # For demonstration, we'll simulate.
    #     sock.close()
    # except bluetooth.BluetoothError as e:
    #     print(f"Could not connect to {device_address} to get RSSI: {e}")
    #     # Fallback to simulation if connection fails
    #     pass

    # Simulate RSSI fluctuation for demonstration purposes
    # In a real app, this would be the actual reading from the Aura Ring
    import random
    # Simulate RSSI fluctuating around a "connected" state (-40 to -70)
    # and occasionally dropping below the threshold to trigger a lock.
    connected_rssi = random.randint(-65, -40)
    disconnected_rssi = random.randint(-95, -75)

    # Introduce a chance for the "device" to "move away"
    if random.random() < 0.2: # 20% chance to simulate moving away
        return disconnected_rssi
    else:
        return connected_rssi

def main():
    print("--- Aura Ring Backend Simulator ---")

    # 1. Discover devices
    devices = discover_bluetooth_devices()
    if not devices:
        print("No devices found. Exiting.")
        sys.exit(1)

    # 2. Select Aura Ring device
    aura_ring_address, aura_ring_name = select_device(devices)

    # 3. Select RSSI sensitivity
    rssi_threshold = select_sensitivity()

    print(f"\nMonitoring RSSI for {aura_ring_name} ({aura_ring_address}).")
    print(f"Screen will lock if RSSI drops below {rssi_threshold} dBm.")
    print("Press Ctrl+C to stop monitoring.")

    screen_locked = False

    try:
        while True:
            current_rssi = get_current_rssi(aura_ring_address)
            print(f"Current RSSI: {current_rssi} dBm")

            if current_rssi < rssi_threshold and not screen_locked:
                print(f"RSSI ({current_rssi} dBm) is below threshold ({rssi_threshold} dBm).")
                if lock_screen():
                    screen_locked = True
                else:
                    print("Could not lock screen. Continuing to monitor.")
            elif current_rssi >= rssi_threshold and screen_locked:
                print(f"RSSI ({current_rssi} dBm) is back above threshold. Screen is still locked.")
                # In a real scenario, you'd have a biometric unlock mechanism here.
                # For this simulation, we assume manual unlock after the event.
                # To re-enable locking after manual unlock, you might need a separate event.
                # For now, once locked, it stays locked in the simulation until restart.
                pass # Keep screen_locked as True to prevent repeated lock attempts

            time.sleep(POLLING_INTERVAL)

    except KeyboardInterrupt:
        print("\nMonitoring stopped by user.")
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()

To install pybluez: pip install pybluez
--- Aura Ring Backend Simulator ---

Searching for Bluetooth devices (this may take a few seconds)...
Bluetooth library not available, simulating device discovery.

Found the following Bluetooth devices:
1. Simulated Aura Ring 1 (AA:BB:CC:DD:EE:F1)
2. Simulated Bluetooth Device (AA:BB:CC:DD:EE:F2)
Enter the number of your Aura Ring device: 1
Selected Aura Ring: Simulated Aura Ring 1 (AA:BB:CC:DD:EE:F1)

Select RSSI Sensitivity Level (screen locks when signal is weaker than this):
1. 1 (Very Low - locks far away) (Threshold: -90 dBm)
2. 2 (Low) (Threshold: -80 dBm)
3. 3 (Medium) (Threshold: -70 dBm)
4. 4 (High) (Threshold: -60 dBm)
5. 5 (Very High - locks very close) (Threshold: -50 dBm)
Enter the number for your desired sensitivity: 4
Selected sensitivity: 4 (High) (Threshold: -60 dBm)

Monitoring RSSI for Simulated Aura Ring 1 (AA:BB:CC:DD:EE:F1).
Screen will lock if RSSI drops below -60 dBm.
Press Ctrl+C to stop monitoring.
Current RSSI: -50 dB